In [9]:
import pandas as pd
with open("w5_e30.txt") as f:
    rawtxt = f.read().replace("\x1b[94m","").replace("\x1b[91m","").replace("\x1b[0m","").replace("\x1b[92m","").split("\n")

In [11]:
rawtxt

['2020/03/12 13:54:16: PARAMS :  experiment_1_SNLI_W5d0.001init_rnn_cellNonLid_SENThyp_INPhyp_BIAShyp_FFNNhyp5id_dsq_MLRhyp_rsgd_lrW0.1_lrFF0.01_prje1e-05_bs64_burnn__13:54:1612M',
 '2020/03/12 13:54:29: Loading train - val - test data',
 '2020/03/12 13:54:32: Training data size: 549367',
 '2020/03/12 13:54:32: Class 1 has 33.3868 percent samples',
 '2020/03/12 13:54:32: Class 0 has 66.6132 percent samples',
 '2020/03/12 13:54:32: Validation data size: 9842',
 '2020/03/12 13:54:32: Class 1 has 34.2207 percent samples',
 '2020/03/12 13:54:32: Class 0 has 65.5964 percent samples',
 '2020/03/12 13:54:32: Test data size: 9824',
 '2020/03/12 13:54:32: Class 1 has 34.2834 percent samples',
 '2020/03/12 13:54:32: Class 0 has 65.7166 percent samples',
 '2020/03/12 13:54:38: Epoch: 0',
 '2020/03/12 13:55:02: For test: ==>  glN_predN = 3410.0; glE_predN = 1482.0; glN_predE = 3046.0; glE_predE = 1886.0',
 '2020/03/12 13:55:02: CURRENT val accuracy: 0.5421 ; test accuracy:  0.5391 ',
 '2020/03/12 

In [47]:
df = pd.DataFrame(columns=['Epoch','val_acc','test_acc'])
df_loss = pd.DataFrame(columns=['curr_loss','sec_per_sent'])
for i in range(len(rawtxt)):
    if "Epoch" in rawtxt[i]:
        epoch =  int(rawtxt[i][-2:])
        val_test_list = rawtxt[i+2].split(" ")
        valac = float(val_test_list[5])
        testac = float(val_test_list[10])
        df = df.append({
            'Epoch':epoch,
            'val_acc':valac,
            'test_acc':testac
        }, ignore_index=True)
    if "curr_loss" in rawtxt[i]:
        loss_list = rawtxt[i].split(" ")
        curr_loss = float(loss_list[7])
        sec_per_sent = float(loss_list[-1])
        df_loss = df_loss.append({
            'curr_loss':curr_loss,
            'sec_per_sent':sec_per_sent
        }, ignore_index=True)
    

ValueError: could not convert string to float: '0.5166;'

In [45]:
df

,Epoch,val_acc,test_acc
0,0.0,0.5421,0.5391
1,1.0,0.7546,0.7488
2,2.0,0.7611,0.7587
3,3.0,0.7684,0.7626
4,4.0,0.7676,0.7589
5,5.0,0.7683,0.7658
6,6.0,0.7758,0.7716
7,7.0,0.7721,0.7693
8,8.0,0.7724,0.7710
9,9.0,0.7742,0.7693


In [46]:
df_loss

,curr_loss,sec_per_sent
0,0.5166;,0.0028
1,0.4528;,0.0029
2,0.5393;,0.0029
3,0.4240;,0.0030
4,0.4315;,0.0029
...,...,...
119,0.5550;,0.0027
120,0.5024;,0.0026
121,0.3660;,0.0026
122,0.4597;,0.0027
